In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
activos = pd.read_csv("../datos/activos_nacionalidad_sexo_edad.csv")
empleo = pd.read_csv("../datos/tasa_de_empleo_anual.csv",sep= ';')
delitos = pd.read_csv("../datos/delitos_nacionalidad.csv")
condenados = pd.read_csv("../datos/condenados_sexo_nacionalidad_numero_delitos.csv")
ocupados = pd.read_csv("../datos/ocupados_nacionalidad_sector.csv")
poblacion = pd.read_csv("../datos/poblacion_pais_nacimiento.csv")
salario = pd.read_csv("../datos/salarios_jornada_nacionalidad_decil.csv")
delito_sexual = pd.read_csv("../datos/delitos_sexuales_nacionalidad.csv",sep= ';')
paro = pd.read_csv("../datos/paro_por_nacionalidad.csv",sep= ';')
no_estudia_ni_trabaja = pd.read_csv("../datos/no_estudian_ni_trabajan_por_nacionalidad.csv",sep= ';')
riesgo_pobreza = pd.read_csv("../datos/riesgo_de_pobreza_nacionalidad.csv",sep= ';')

In [13]:
def clean_numeric_column(series):
    return pd.to_numeric(series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False), errors='coerce')

for df in [activos, empleo, delitos, condenados, ocupados, poblacion, salario,delito_sexual,paro,no_estudia_ni_trabaja,riesgo_pobreza]:
    df.columns = df.columns.str.replace(r'^\ufeff', '', regex=True)
    if 'Total' in df.columns:
        df['Total'] = clean_numeric_column(df['Total'])

In [14]:
empleo

,Nacionalidad,Periodo,Total
0,Total,2024,71.37
1,Total,2023,70.55
2,Total,2022,69.30
3,Total,2021,67.48
4,Total,2020,65.73
5,Total,2019,68.03
6,Total,2018,67.01
7,Total,2017,65.55
8,Total,2016,63.87
9,Total,2015,61.97


In [7]:
delitos


,Tipo de Delito: Nivel 1,Tipo de Delito: Nivel 2,Tipo de Delito: Nivel 3,Tipo de Delito: Nivel 4,Nacionalidad,Periodo,Total
0,A Delitos,NaN,NaN,NaN,Total,2023,403194.0
1,A Delitos,NaN,NaN,NaN,Total,2022,426416.0
2,A Delitos,NaN,NaN,NaN,Total,2021,410842.0
3,A Delitos,NaN,NaN,NaN,Total,2020,311271.0
4,A Delitos,NaN,NaN,NaN,Total,2019,412571.0
...,...,...,...,...,...,...,...
6705,A Delitos,R Resto de delitos,NaN,NaN,De Oceanía,2017,0.0
6706,A Delitos,R Resto de delitos,NaN,NaN,De Oceanía,2016,0.0
6707,A Delitos,R Resto de delitos,NaN,NaN,De Oceanía,2015,10.0
6708,A Delitos,R Resto de delitos,NaN,NaN,De Oceanía,2014,0.0


In [8]:
ocupados

,Sexo,Sector económico,Nacionalidad,Periodo,Total
0,Ambos sexos,Total,Total,2023,21005.5
1,Ambos sexos,Total,Total,2022,20390.6
2,Ambos sexos,Total,Total,2021,19773.6
3,Ambos sexos,Total,Española,2023,17166.9
4,Ambos sexos,Total,Española,2022,16948.5
...,...,...,...,...,...
355,Mujeres,Servicios,Extranjera: América Latina,2022,485.4
356,Mujeres,Servicios,Extranjera: América Latina,2021,448.6
357,Mujeres,Servicios,Extranjera: Resto del mundo y apátrida,2023,197.9
358,Mujeres,Servicios,Extranjera: Resto del mundo y apátrida,2022,168.8
